In [12]:
import networkx as nx
from langchain_openai import OpenAI
from langchain.chains.graph_qa.base import GraphQAChain
from neo4j import GraphDatabase

In [21]:
import vertexai
from vertexai.generative_models import GenerativeModel
from neo4j import GraphDatabase

def generate_cypher_query():
    # Initialize Vertex AI with your project details
    vertexai.init(project="gen-lang-client-0447891830", location="us-east1")

    # Load the generative model
    model = GenerativeModel("gemini-1.5-flash-001")

    # Define the context for generating Cypher queries
    context =  """
    You are an expert in querying knowledge graphs. Our graph has been constructed using data from three main sources: Assurance, Carte, and Pack. Each source is structured similarly, with nodes representing different entities and relationships connecting them.

    ### Instructions:
    - When generating a Cypher query, ensure that the output is a plain text Cypher query with no additional formatting, code block delimiters, or Markdown.
    - The query should start directly with the Cypher command (e.g., `MATCH`, `RETURN`)
    - Do not include any additional text, explanations, or comments in the output.
   ### Nodes:
    1. **Assurance**: Represents insurance products with properties:
       - `productId` (String): Unique identifier for the assurance product.
       - `productType` (String): The type of the assurance.
       - `productName` (String): The name of the assurance product.

    2. **Target**: Represents target descriptions for products with a single property:
       - `description` (String): Description of the target audience or purpose.

    3. **Formula**: Represents a formula related to a product with properties:
       - `composite_key` (String): A unique key combining name and price.
       - `name` (String): Name of the formula.
       - `price` (Float): Price associated with the formula.
       - `currency` (String): The currency in which the price is denominated.

    4. **Feature**: Represents features associated with a formula with a property:
        - `description` (String): Description of the feature.

    ### Relationships:
    - `HAS_TARGET`: Connects an `Assurance` node to one or more `Target` nodes.
    - `HAS_FORMULA`: Connects an `Assurance` node to one or more `Formula` nodes.
    - `HAS_FEATURE`: Connects a `Formula` node to one or more `Feature` nodes.

    EXAMPLES:
    ** MATCH (p:Assurance {productName: "Product Name"})-[:HAS_TARGET]->(t:Target)
    RETURN t.description;
    ** MATCH (a:Assurance)-[r:HAS_FORMULA|HAS_TARGET]->(connectedNode)
    RETURN a.productName, type(r), connectedNode
    ### Nodes:
    1. **Pack**: Represents product packs with properties:
       - `productId` (String): Unique identifier for the pack.
       - `productType` (String): The type of the pack.
       - `productName` (String): The name of the pack.

    2. **Target**: Represents target descriptions for products with a single property:
       - `description` (String): Description of the target audience or purpose.

    3. **Formula**: Represents a formula related to a product with properties:
       - `composite_key` (String): A unique key combining name and price.
       - `name` (String): Name of the formula.
       - `price` (Float): Price associated with the formula.
       - `currency` (String): The currency in which the price is denominated.

    4. **Feature**: Represents features associated with a formula with a property:
        - `description` (String): Description of the feature.

    ### Relationships:
    - `HAS_TARGET`: Connects a `Pack` node to one or more `Target` nodes.
    - `HAS_FORMULA`: Connects a `Pack` node to one or more `Formula` nodes.
    - `HAS_FEATURE`: Connects a `Formula` node to one or more `Feature` nodes.

    EXAMPLES:
    ** MATCH (p:Pack {productName: "Pack Business"})-[:HAS_TARGET]->(t:Target)
    RETURN t.description;
    ** MATCH (p:Pack)-[r:HAS_FORMULA|HAS_TARGET]->(connectedNode)
    RETURN p.productName, type(r), connectedNode
     ### Nodes:
    1. **Product**: Represents products (cards) with properties:
       - `productId` (String): Unique identifier for the product.
       - `productType` (String): The type of the product.
       - `productName` (String): The name of the product.

    2. **Target**: Represents target descriptions for products with a single property:
       - `description` (String): Description of the target audience or purpose.

    3. **Pricing**: Represents the pricing details associated with products, having properties:
       - `price` (Float): The price of the product.
       - `currency` (String): The currency in which the price is denominated.

    4. **ExtraCosts**: Represents additional costs related to a product with properties:
       - `costs` (String): Comma-separated list of extra costs.

    5. **Characteristic**: Represents product characteristics with a property:
       - `title` (String): Title of the characteristic.

    6. **Detail**: Represents detailed information about a characteristic with properties:
       - `composite_key` (String): A unique key combining head and content.
       - `head` (String): A header or title of the detail.
       - `content` (String): The content or body of the detail.

    ### Relationships:
    - `HAS_TARGET`: Connects a `Product` node to one or more `Target` nodes.
    - `HAS_PRICING`: Connects a `Product` node to its `Pricing` node.
    - `HAS_EXTRA_COSTS`: Connects a `Pricing` node to its `ExtraCosts` node.
    - `HAS_CHARACTERISTIC`: Connects a `Product` node to one or more `Characteristic` nodes.
    - `INCLUDES_DETAIL`: Connects a `Characteristic` node to one or more `Detail` nodes.

    EXAMPLES:
    ** MATCH (p:Product {productName: "Product Name"})-[:HAS_TARGET]->(t:Target)
    RETURN t.description;
    ** MATCH (p:Product)-[r:HAS_PRICING|HAS_CHARACTERISTIC]->(connectedNode)
    RETURN p.productName, type(r), connectedNode
    when retriving data always get the product details!

    """
  

    # Define the natural language prompt for the specific query you want to generate
    prompt = "give details about carte cashback"

    # Combine the context with the prompt
    full_input = context + "\n" + prompt

    # Generate the Cypher query based on the combined context and prompt
    response = model.generate_content(
        [full_input],
        generation_config=generation_config,
        stream=False,
    )

    # Extract the generated Cypher query
    cypher_query = response.text.strip()

    # Remove any potential code block markers if they exist
    if cypher_query.startswith("```cypher"):
        cypher_query = cypher_query.replace("```cypher", "").replace("```", "").strip()

    return cypher_query

def execute_cypher_query(cypher_query):
    # Connect to the Neo4j database
    driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "walid123"))

    with driver.session() as session:
        # Execute the Cypher query and return the results
        result = session.run(cypher_query)
        return [record.data() for record in result]

def format_results_for_user(results):
    # Initialize Vertex AI for generating a human-readable response
    model = GenerativeModel("gemini-1.5-flash-001")

    # Format the results into a text description for the user
    prompt = f"Given the following data from a Neo4j database query: {results}, create a human-readable explanation in paragraph format the answer should be in a context where you are talking to one of attijariwafabank clients and he is looking for informations that you are providing most question are deterministic questions."

    response = model.generate_content(
        [prompt],
        generation_config=generation_config,
        stream=False,
    )

    return response.text.strip()

def main():
    # Generate the Cypher query using the LLM
    cypher_query = generate_cypher_query()
    print(f"Generated Cypher Query:\n{cypher_query}\n")

    # Execute the generated Cypher query on the Neo4j database
    results = execute_cypher_query(cypher_query)

    # Generate a human-readable explanation of the results
    human_readable_output = format_results_for_user(results)

    # Print the formatted results
    print("Human-Readable Output:")
    print(human_readable_output)

# Define the generation configuration
generation_config = {
    "max_output_tokens": 1024,
    "temperature": 0,
    "top_p": 0.95,
}

if __name__ == "__main__":
    main()


Generated Cypher Query:
MATCH (p:Carte {productName: "Carte Cashback"})-[:HAS_TARGET]->(t:Target)
RETURN t.description

Human-Readable Output:
Based on the information from our database, it seems you're interested in understanding the different types of accounts offered by Attijariwafa Bank.  The first type, "Entreprises et Professionnels, détenteurs d’un compte commercial," is designed specifically for businesses and professionals who need a commercial account. This type of account is ideal for managing business transactions, receiving payments, and making payments to suppliers. The second type, "Clients Privilège Particuliers et Professionnels, détenteurs d’un compte chéques," is a privileged account for both individuals and professionals who need a checking account. This account offers a range of benefits and services tailored to meet the needs of our valued clients.
